In [5]:
import requests
import json
from urllib.parse import urljoin
from pathlib import Path


In [6]:
base_url = 'http://localhost:8000/'

In [7]:
def create_url_endpoint(endpoint: str, base_url: str = base_url)  -> str:
    return urljoin(base_url, endpoint)

In [8]:
path_data = Path.cwd()/'data_raw'
path_json_init =list( path_data.glob('*.json'))
print('Основной json для инициализации хранилища, _1  срез первые 16 текстов  _2  срез полседние 16 текстов')
print('Для тестов и отладки')
path_json_init

Основной json для инициализации хранилища, _1  срез первые 16 текстов  _2  срез полседние 16 текстов
Для тестов и отладки


[WindowsPath('C:/Users/dmitr/project_loc/Loymax/data_raw/RuBQ_2.0_paragraphs.json'),
 WindowsPath('C:/Users/dmitr/project_loc/Loymax/data_raw/RuBQ_2.0_paragraphs_1.json'),
 WindowsPath('C:/Users/dmitr/project_loc/Loymax/data_raw/RuBQ_2.0_paragraphs_2.json')]

### 1. For indexer

#### 1.1.  Эндпоинт  инициализации хранилища

# 'name_json_init' : 'RuBQ_2.0_paragraphs_1.json'  - тестовый вариант инициализации, пропсан  в config

In [48]:
endpoint ='create_index'
url = urljoin(base_url, endpoint)
response = requests.post(url)

print(response.status_code)
print(response.json())

200
{'status': 'Index created successfully'}


In [9]:
endpoint ='create_index'
url = urljoin(base_url, endpoint)
response = requests.post(url)

print(response.status_code)
print(response.json())

200
{'status': 'Index created successfully'}


#### 1.2. Эндпоинт для добавления текстов в индекс из JSON-файла по указанному пути

In [19]:
path_json_add = str(path_json_init[2])
path_json_add

'C:\\Users\\dmitr\\project_loc\\Loymax\\data_raw\\RuBQ_2.0_paragraphs_2.json'

In [49]:
endpoint  = 'add_index' # принимает json строку
url = create_url_endpoint(endpoint)

response = requests.post(
    url, json.dumps(path_json_add),
    headers={"Content-Type": "application/json"}
)

print(response.status_code)
print(response.json())

200
{'status': 'Added texts from file C:\\Users\\dmitr\\project_loc\\Loymax\\data_raw\\RuBQ_2.0_paragraphs_2.json to index'}


#### 1.3. Эндпоинт для поиска схожих текстов. Принимает строку (json строка) и возвращает список расстояний и найденных текстов

In [54]:
endpoint = 'search_texts'
url = create_url_endpoint(endpoint)
text ='''
В первых двух турах ЦСКА потерпел два поражения, 
сначала в гостевом матче с «Черноморцем» 2:0, затем в домашнем матче против самарского клуба «Крылья Советов» 3:0.
'''
response = requests.post(
    url, json.dumps(text.strip()),
    headers={"Content-Type": "application/json"}
)

print(response.status_code)

print(response.json()["similarity_scores"])

200
[0.6621801257133484, 0.8026571273803711, 0.8354564905166626]


In [55]:
print(  response.json()["retrieved_texts"])

['В первом сезоне в составе Континентальной хоккейной лиги ЦСКА выиграл дивизион Тарасова, но в плей-офф с трудом обыграл «Ладу» (3-2 по сумме встреч) и всухую проиграл «Динамо» (0-3). В конце сезона тренерский тандем Быков-Захаркин покинул команду, аргументировав своё решение желанием сосредоточиться на работе в сборной России, однако уже через несколько недель подписали контракт с командой «Салават Юлаев», таким образом продолжив совмещать работу в сборной и в клубе.', '21 февраля 2001 года было объявлено о смене собственника армейского клуба. Новыми акционерами стали компания «АВО-Капитал» — 51 % акций, Министерство Обороны РФ и английская фирма «Blue Castle Enterprises Limited», а президентом клуба стал российский предприниматель Евгений Леннорович Гинер. К этому времени команду покинул ряд футболистов, считавшихся в прошлом сезоне основными: Хомуха, Бычков, Окрошидзе и Шишкин. До закрытия трансферного окна новому руководству удалось приобрести трёх футболистов, реально усиливших с

#### 1.3. Удаляет файлы сотояния хранилища, после возможна инициализация на новых данных

In [56]:
endpoint = 'delete_index_files'
url = create_url_endpoint(endpoint)
response = requests.delete(url)

print(response.status_code)
print(response.json())



200
{'status': 'Index files deleted'}


### 2. For answer

In [57]:
base_url_2 = 'http://localhost:8001/'

#### 2.1. Генерация ответа на вопрос.

In [58]:
endpoint ='answer_question'
url = create_url_endpoint(endpoint, base_url=base_url_2)

In [77]:
text = 'Расскажи про спортивный клуб  ЦСКА и его историю'

response = requests.post(
    url, json.dumps(text.strip()),
    headers={"Content-Type": "application/json"}
)
answer = response.json()

print(response.status_code)
print(answer['query'])
print(answer['answer'].strip())
print(answer['result_indexer'])

200
Расскажи про спортивный клуб  ЦСКА и его историю
Ответ: ЦСКА — советский и российский профессиональный хоккейный клуб из Москвы, основан в 1946 году под названием ЦДКА. С 1951 года назывался ЦДСА, с 1954 года — ЦСК МО, и с 1959 года носит название ЦСКА. Выступает в Континентальной хоккейной лиге.
{'similarity_scores': [0.8233098387718201], 'retrieved_texts': ['ЦСКА — советский и российский профессиональный хоккейный клуб из Москвы, выступающий в Континентальной хоккейной лиге. Основан в 1946 году под названием ЦДКА (Центральный дом Красной Армии). В 1951 году переименован в ЦДСА (Центральный дом Советской Армии), а в 1954 в ЦСК МО (Центральный спортивный клуб Министерства обороны), под которым выступал до 1959 года, и с тех пор носит название ЦСКА (Центральный Спортивный Клуб Армии).']}


In [75]:
text = 'Расскажи про АВО-Капитал'
response = requests.post(
    url, json.dumps(text.strip()),
    headers={"Content-Type": "application/json"}
)
answer = response.json()

print(response.status_code)
print(answer['query'])
print(answer['answer'].strip())
print(answer['result_indexer'])

200
Расскажи про АВО-Капитал
"Я не нашел информации по вашему запросу в доступных источниках."
{'similarity_scores': [1.551884412765503], 'retrieved_texts': ['ЦСКА — советский и российский профессиональный хоккейный клуб из Москвы, выступающий в Континентальной хоккейной лиге. Основан в 1946 году под названием ЦДКА (Центральный дом Красной Армии). В 1951 году переименован в ЦДСА (Центральный дом Советской Армии), а в 1954 в ЦСК МО (Центральный спортивный клуб Министерства обороны), под которым выступал до 1959 года, и с тех пор носит название ЦСКА (Центральный Спортивный Клуб Армии).']}
